In [ ]:
import os,sys

# torch 
import torch # pytorch package, allows using GPUs
import torch.nn.functional as F # implements forward and backward definitions of an autograd operation
from torch import nn, optim
from torch.nn.modules import Module
from torchvision import datasets # load data
from torch.autograd import Variable
import torch.optim as optim
# fix seed
seed=12
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
class plain_model(nn.Module):
    def __init__(self, lin_layers, activation = nn.ReLU(), out_activation = nn.LogSoftmax(dim=1), device=None):
        super().__init__()
        ########### setting up conv layers ###########
        self.layers = nn.ModuleList()
        self.layers.append(nn.Conv2d(1, 32, kernel_size=5))
        self.layers.append(activation)
        self.layers.append(nn.Conv2d(32, 64, kernel_size=5))
        self.layers.append(activation)
        self.layers.append(nn.MaxPool2d(2, strides = 2))
        self.layers.append(nn.Conv2d(64, 64, kernel_size=1))
        self.layers.append(activation)
        self.layers.append(nn.MaxPool2d(2, strides = 2))
        self.layers.append(nn.Flatten())

        layers_full = [input_size] + lin_layers
        
        ################## FFN ##################  
        # variable number of hidden layers and neurons, controlled by 
        for i, layer in enumerate(layers):
            self.layers.append(nn.Linear(layers_full[i], layer))
            if activation is not None:
                assert isinstance(activation, Module), \
                self.layers.append(activation)
            self.layers.append(nn.BatchNorm2d())

        self.layers.append(nn.Linear(layers[-1], 2))
        
        # only one dropout layer after all BN layers inspired by https://arxiv.org/pdf/1801.05134.pdf (also we have a very low amount of training data)
        self.layers.append(nn.Dropout(.25))

        self.layers.append(out_activation)
        
        if device = None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = device

        self.to(self.device)

    def forward(self, input_data, train=True):
        x = test_tensor = torch.cat((input_data[:, 0, :,:], input_data[:,1, :, :]), 2)
        for layer in self.layers:
            x = layer(x)
        return x